# Trainer

In [1]:
import torch
import torch.nn as nn

## Prepare Dataset

In [2]:
from torchhk.datasets import *

In [3]:
mnist = Datasets("MNIST", root='./data',
                 transform_train=transforms.ToTensor(), 
                 transform_test=transforms.ToTensor())

train_loader, _ = mnist.get_loader(batch_size=128)

Data Loaded!
Train Data Length : 60000
Test Data Length : 10000


## Prepare Model

In [4]:
from model import CNN
model = CNN().cuda()

## Train Model

In [5]:
from torchhk.trainer import Trainer

In [6]:
######################### Important Attributes ###########################
# self.model : model
# self.device : device where model is
# self.optimizer : optimizer
# self.scheduler : scheduler (* Automatically Updated)
# self.max_epoch : total number of epochs
# self.max_iter : total number of iterations
# self.epoch : current epoch (* Automatically Updated)
# self.iter : current iter (* Automatically Updated)
# self.record_keys : items to record == items returned by do_iter
#########################################################################

class BaseTrainer(Trainer):
    def __init__(self, model, **kwargs):
        super(BaseTrainer, self).__init__("BaseTrainer", model, **kwargs)
        # Set Records (* Must be same as the items returned by do_iter)
        self.record_keys = ["Loss", "Acc"]
    
    # Override Do Iter
    def _do_iter(self, train_data):
        images, labels = train_data
        X = images.to(self.device)
        Y = labels.to(self.device)

        pre = self.model(X)
        cost = nn.CrossEntropyLoss()(pre, Y)

        self.optimizer.zero_grad()
        cost.backward()
        self.optimizer.step()

        _, pre = torch.max(pre.data, 1)
        total = pre.size(0)
        correct = (pre == Y).sum()
        cost = cost.item()
        
        return cost, 100*float(correct)/total
    
    # Override Update Scheduler
    def _update_scheduler(self):
        self.scheduler.step()

In [7]:
trainer = BaseTrainer(model)

In [8]:
trainer.train(train_loader=train_loader, max_epoch=4,
              optimizer="SGD(lr=0.01, momentum=0.9)",
              scheduler="MultiStepLR(milestones=[4, 6], gamma=0.1)", scheduler_type="Epoch",
              # Easy Scheduler String for the most used:
              ## Step(milestones=[2, 4], gamma=0.1)
              ## Cyclic(base_lr=0, max_lr=0.3)
              ## Cosine
              save_type="Epoch", save_path="./_models/", save_overwrite=False,
              record_type="Epoch")

[BaseTrainer]
Training Information.
-Epochs: 4
-Optimizer: SGD (
Parameter Group 0
    dampening: 0
    initial_lr: 0.01
    lr: 0.01
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)
-Scheduler: <torch.optim.lr_scheduler.MultiStepLR object at 0x000001ACC93BC080>
-Save Path: ./_models/
-Save Type: Epoch
-Record Type: Epoch
-Device: cuda:0
------------------------------------               
Epoch   Loss     Acc       lr       
1       0.6329   79.6191   0.0100   
------------------------------------
2       0.0898   97.1738   0.0100                  
------------------------------------
3       0.0579   98.2255   0.0100                  
------------------------------------
4       0.0462   98.5260   0.0100                  
------------------------------------
Total Epoch: 4
Time Elapsed: 0:00:30.938809
Min(epoch)/Max(epoch): 
-Loss: 0.0462(4)/0.6329(1)
-Acc: 79.6191(1)/98.5260(4)
-lr: 0.0100(1)/0.0100(1)
------------------------------------


In [9]:
trainer.train(train_loader=train_loader, max_epoch=8, start_epoch=4,
              optimizer="SGD(lr=0.01, momentum=0.9)",
              scheduler="MultiStepLR(milestones=[4, 6], gamma=0.1)", scheduler_type="Epoch",
              # or scheduler="Step([4, 6], 0.1)"
              save_type="Epoch", save_path="_models/", save_overwrite=True,
              record_type="Epoch")

[BaseTrainer]
Training Information.
-Epochs: 8
-Optimizer: SGD (
Parameter Group 0
    dampening: 0
    initial_lr: 0.01
    lr: 0.01
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)
-Scheduler: <torch.optim.lr_scheduler.MultiStepLR object at 0x000001ACC93BC4A8>
-Save Path: _models/
-Save Type: Epoch
-Record Type: Epoch
-Device: cuda:0
Progress: \ [0:00:00.015596/it]                    

C:\Users\slcf\Anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:122: UserWarning:

Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate



------------------------------------               
Epoch   Loss     Acc       lr       
5       0.0272   99.1904   0.0010   
------------------------------------
6       0.0240   99.3005   0.0010                  
------------------------------------
7       0.0219   99.3840   0.0001                  
------------------------------------
8       0.0215   99.3857   0.0001                  
------------------------------------
Total Epoch: 8
Time Elapsed: 0:00:29.846119
Min(epoch)/Max(epoch): 
-Loss: 0.0215(8)/0.0272(5)
-Acc: 99.1904(5)/99.3857(8)
-lr: 0.0001(7)/0.0010(5)
------------------------------------


In [10]:
trainer.save_all("final")

Saving Model
...Saved as pth to final.pth !
Saving Records
...Saved as csv to final.csv !
